In [17]:
from collections import Counter
from pathlib import Path
import os
import re
import shutil

import pandas as pd
import tifffile
from tifffile import TiffFile
from ome_types import from_xml

In [2]:
import multiplex_imaging_pipeline.multiplex_imaging_pipeline as mip

2024-08-20 16:10:31.047000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/lib:/usr/lib64:/usr/local/lib:/usr/lib64:
2024-08-20 16:10:31.047086: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
output_dir = Path(f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc')
output_dir.mkdir(parents=True, exist_ok=True)

In [4]:
df = pd.read_csv('/diskmnt/Projects/Users/estorrs/codex_specimens.txt',
                     sep='\t')
df.index = [f'{name}_{specimen}' for name, specimen in zip(df['omero_run_name'], df['specimen'])]
df

,run_name,omero_run_name,date,case,specimen,bbox,filepath
2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U14,20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1,2024-07-23_HT891Z1,20240723,HT891Z1,HT891Z1-S2H3Fp1_U14,"561,12824,560,13960",/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...
2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U31,20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1,2024-07-23_HT891Z1,20240723,HT891Z1,HT891Z1-S2H3Fp1_U31,"15747,28742,1616,15016",/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...
2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U43,20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1,2024-07-23_HT891Z1,20240723,HT891Z1,HT891Z1-S2H3Fp1_U43,"32559,45553,1210,14935",/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...
2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U58,20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1,2024-07-23_HT891Z1,20240723,HT891Z1,HT891Z1-S2H3Fp1_U58,"48639,62039,1616,14122",/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...
2024-07-23_HT891Z1_2_HT891Z1-S2H3Fp1_U79,20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex2,2024-07-23_HT891Z1_2,20240723,HT891Z1,HT891Z1-S2H3Fp1_U79,"0,11726,521,12910",/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...
...,...,...,...,...,...,...,...
2021-10-19_HT339B1_HT339B1-S1H3A1,20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,20211019,HT339B1,HT339B1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...
2021-10-08_HT323B1_HT323B1-S1H3A4,20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,20211008,HT323B1,HT323B1-S1H3A4,NaN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...
2021-09-21_HT293N1_HT293N1-S1H3A1,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...
2021-09-21_HT293N1_2_HT293N1-S1H3A1,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1_2,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...


In [5]:
list(df.index)

['2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U14',
 '2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U31',
 '2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U43',
 '2024-07-23_HT891Z1_HT891Z1-S2H3Fp1_U58',
 '2024-07-23_HT891Z1_2_HT891Z1-S2H3Fp1_U79',
 '2024-07-23_HT891Z1_2_HT891Z1-S2H3Fp1_U93',
 '2024-07-23_HT891Z1_2_HT891Z1-S2H3Fp1_U103',
 '2024-07-23_HT891Z1_2_HT891Z1-S2H3Fp1_U113',
 '2024-07-20_HT891Z1_HT891Z1-S2H3Fp1_U40',
 '2024-07-20_HT891Z1_HT891Z1-S2H3Fp1_U41',
 '2024-07-20_HT891Z1_HT891Z1-S2H3Fp1_U45',
 '2024-07-20_HT891Z1_HT891Z1-S2H3Fp1_U46',
 '2024-07-20_HT891Z1_2_HT891Z1-S2H3Fp1_U108',
 '2024-07-20_HT891Z1_2_HT891Z1-S2H3Fp1_U109',
 '2024-07-20_HT891Z1_2_HT891Z1-S2H3Fp1_U110',
 '2024-07-20_HT891Z1_2_HT891Z1-S2H3Fp1_U111',
 '2024-07-05_HT704B1_3_HT704B1_U76',
 '2024-07-05_HT704B1_3_HT704B1_U82',
 '2024-07-05_HT704B1_3_HT704B1_U85',
 '2024-07-05_HT704B1_3_HT704B1_U91',
 '2024-07-05_HT704B1_HT704B1_U43',
 '2024-07-05_HT704B1_HT704B1_U53',
 '2024-07-05_HT704B1_HT704B1_U60',
 '2024-07-05_HT704B1_HT704B1_U68',
 '

In [6]:
run_names = [
    '2024-06-07_S053TL__S060TRF_S060TRF',
    '2024-06-07_S053TL__S060TRF_S053TL',
    '2024-06-07_S062TLF__S109TRF_S062TLF',
    '2024-06-07_S062TLF__S109TRF_S109TRF',
    '2024-06-06_S026TL__S074TLF_S074TLF',
    '2024-06-06_S026TL__S074TLF_S026TL',
    '2024-06-06_S078TLF__S099TRF_S078TLF',
    '2024-06-06_S078TLF__S099TRF_S099TRF',
    '2024-05-29_S094TRF__S106TLF_S094TRF',
    '2024-05-29_S094TRF__S106TLF_S106TLF',
    '2024-05-29_S041TRF__S100TLF_S100TLF',
    '2024-05-29_S041TRF__S100TLF_S041TRF',
    '2024-05-17_S046TLF__S125T2F_S125T2F',
    '2024-05-17_S046TLF__S125T2F_S046TLF',
    '2024-05-17_S043TL__S118T2F_S043TL',
    '2024-05-17_S043TL__S118T2F_S118T2F',
    '2024-05-16_S003T__S058TLF_S058TLF',
    '2024-05-16_S003T__S058TLF_S003T',
    '2024-05-16_S012TL__S040TL_S040TL',
    '2024-05-16_S012TL__S040TL_S012TL',
]

In [7]:
def get_ome_tiff_channels(filepath):   
    tif = TiffFile(filepath)
    ome = from_xml(tif.ome_metadata)
    im = ome.images[0]
    return [c.name for c in im.pixels.channels]

def get_common_channels(filepaths, channel_mapping=None):
    channel_mapping = channel_mapping if channel_mapping is not None else {}
    pool = []
    for filepath in filepaths:
        channels = get_ome_tiff_channels(filepath)
        channels = [channel_mapping.get(c, c) for c in channels]
        pool += channels
    counts = Counter(pool)
    channels = sorted([c for c, count in counts.items() if count==len(filepaths)])
    return channels

In [8]:
sid_to_fp = {}
for i, row in df.iterrows():
    if i in run_names:
        sid_to_fp[row['specimen']] = row['filepath']
sid_to_fp.keys()

dict_keys(['S060TRF', 'S053TL', 'S062TLF', 'S109TRF', 'S074TLF', 'S026TL', 'S078TLF', 'S099TRF', 'S094TRF', 'S106TLF', 'S100TLF', 'S041TRF', 'S125T2F', 'S046TLF', 'S043TL', 'S118T2F', 'S058TLF', 'S003T', 'S040TL', 'S012TL'])

In [9]:
get_common_channels(sid_to_fp.values())

['CCL2 (D)',
 'CD11b (D)',
 'CD20',
 'CD31',
 'CD3e',
 'CD4',
 'CD45 (D)',
 'CD68 (D)',
 'CD8',
 'CK7 (D)',
 'CK8/18 (D)',
 'DAPI',
 'E-cadherin (D)',
 'FOXP3',
 'GLUT (D)',
 'HLA-DR',
 'Keratin 14',
 'Ki67',
 'MUC2 (D)',
 'P16 (D-new)',
 'P21 (D)',
 'PAI (D)',
 'Pan-Cytokeratin',
 'Podoplanin',
 'SMA (D)',
 'SOX9 (D)',
 'Vimentine']

In [10]:
nuclei_channels = ['DAPI']
membrane_channels = [
    'Pan-Cytokeratin', 'HLA-DR', 'SMA (D)', 'CD4', 'CD45 (D)', 'CD31', 'E-cadherin', 'CD68 (D)', 'CD3e'
]

In [19]:
for specimen, fp in sid_to_fp.items():
#     if specimen == 'S060TRF':
#         continue
    print(specimen)
    prefix = os.path.join(output_dir, specimen)
    print(prefix)
    mip.segment_ome(fp, prefix, 25000, nuclei_channels, membrane_channels)
    seg_fp = prefix + '_cell_segmentation.tif'
    df, a = mip.get_spatial_features(seg_fp, fp, thresholds={})
    
    a.write_h5ad(os.path.join(output_dir, f'{specimen}_spatial_features.h5ad'))
    df.to_csv(os.path.join(output_dir, f'{specimen}_spatial_features.txt'), sep='\t', index=False)

    # copy ome to output directory
    shutil.copy(fp, output_dir / f'{specimen}.ome.tiff')
#     break

2024-08-21 09:57:37,005 - starting segmentation for /diskmnt/primary/CODEX/HTAN/20240607_Human_CRC_Vanderbilt_S060TRF_S053TL/S053TL__20240607.ome.tiff
2024-08-21 09:57:37,007 - Checking for cached data


S053TL
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S053TL


2024-08-21 09:57:37,257 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-08-21 09:57:37,259 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-08-21 09:57:37,259 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-08-21 09:57:40,054 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-08-21 09:57:47,214 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-08-21 10:34:33,210 - finished segmentation
2024-08-21 10:34:33,216 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S053TL_nuclei_segmentation.tif
2024-08-21 10:34:34,342 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S053TL_cell_segmentation.tif
2024-08-21 10:34:35,813 - extracting /diskmnt/primary/CODEX/HTAN/20240607_Human_CRC_Vanderbilt_S060TRF_S053TL/S053TL__20240607.ome.tiff
2024-08-21 10:34:46,026 - extracting /diskmnt/Projects/Users/estorrs/imaging-

S062TLF
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S062TLF


...

2024-08-22 17:39:13,509 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-08-22 17:39:13,510 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-08-22 17:39:13,511 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-08-22 17:39:16,491 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-08-22 17:39:24,313 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-08-22 18:47:08,212 - finished segmentation
2024-08-22 18:47:08,221 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S040TL_nuclei_segmentation.tif
2024-08-22 18:47:11,950 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S040TL_cell_segmentation.tif
2024-08-22 18:47:15,372 - extracting /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S040TL_S012TL/S040TL__20240516.ome.tiff
2024-08-22 18:47:33,736 - extracting /diskmnt/Projects/Users/estorrs/imaging-a

S012TL
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S012TL


2024-08-22 18:54:05,292 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-08-22 18:54:05,293 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-08-22 18:54:05,294 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-08-22 18:54:08,383 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-08-22 18:54:16,066 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-08-22 19:58:56,238 - finished segmentation
2024-08-22 19:58:56,259 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S012TL_nuclei_segmentation.tif
2024-08-22 19:58:59,899 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S012TL_cell_segmentation.tif
2024-08-22 19:59:04,094 - extracting /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S040TL_S012TL/S012TL__20240516.ome.tiff
2024-08-22 19:59:21,733 - extracting /diskmnt/Projects/Users/estorrs/imaging-a

In [14]:
seg_fp = '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S060TRF_cell_segmentation.tif'
ome_fp = sid_to_fp['S060TRF']
df, a = mip.get_spatial_features(seg_fp, ome_fp, thresholds={})

2024-08-21 09:37:36,191 - extracting /diskmnt/primary/CODEX/HTAN/20240607_Human_CRC_Vanderbilt_S060TRF_S053TL/S060TRF__20240607.ome.tiff
2024-08-21 09:37:59,214 - extracting /diskmnt/Projects/Users/estorrs/imaging-analysis/results/vumc/S060TRF_cell_segmentation.tif
2024-08-21 09:38:01,687 - thresholds detected: [('DAPI', 0.0), ('Podoplanin', 0.0), ('Pan-Cytokeratin', 0.0), ('HLA-DR', 0.0), ('CD8', 0.0), ('SMA (D)', 0.0), ('CD45 (D)', 0.0), ('Ki67', 0.0), ('Vimentine', 0.0), ('GLUT (D)', 0.0), ('Keratin 14', 0.0), ('CD31', 0.0), ('CCL2 (D)', 0.0), ('E-cadherin (D)', 0.0), ('MUC2 (D)', 0.0), ('CD11b (D)', 0.0), ('SOX9 (D)', 0.0), ('CD20', 0.0), ('FOXP3', 0.0), ('P21 (D)', 0.0), ('CK7 (D)', 0.0), ('CD68 (D)', 0.0), ('P16 (D-new)', 0.0), ('CD4', 0.0), ('PAI (D)', 0.0), ('CD3e', 0.0), ('CK8/18 (D)', 0.0)]
2024-08-21 09:38:40,918 - num cells: 339371
2024-08-21 09:42:00,830 - cell typing key is: fraction
2024-08-21 09:42:00,834 - gating cells with the following markers: ['CCL2', 'CD11b', 'CD2

In [15]:
df

,label,row,col,bbox-r1,bbox-c1,bbox-r2,bbox-c2,area,CCL2_fraction,CCL2_intensity,...,SMA_fraction,SMA_intensity,SMA_intensity_scaled,SOX9_fraction,SOX9_intensity,SOX9_intensity_scaled,Vimentine_fraction,Vimentine_intensity,Vimentine_intensity_scaled,cell_type
0,1,411.572159,10344.068030,380,10313,441,10379,2543.0,0.041683,0.096736,...,0.019269,0.019269,0.002593,0.055840,0.061345,0.037674,0.062918,0.063704,0.002129,Epithelial
1,2,411.258678,16104.945455,380,16075,442,16139,2420.0,0.049587,0.109091,...,0.015289,0.015289,0.002058,0.026033,0.028512,0.017510,0.090496,0.091736,0.003066,Treg
2,3,413.149977,19121.131400,396,19058,430,19180,2207.0,0.004078,0.004078,...,0.000000,0.000000,0.000000,0.000453,0.000453,0.000278,0.004984,0.005437,0.000182,Unlabeled
3,4,414.311904,19321.011613,396,19251,433,19388,2411.0,0.010369,0.010369,...,0.001244,0.001244,0.000167,0.000830,0.000830,0.000509,0.006221,0.006221,0.000208,Unlabeled
4,5,414.523202,19511.174014,396,19443,432,19573,2586.0,0.013148,0.013148,...,0.001160,0.001160,0.000156,0.001933,0.001933,0.001187,0.009281,0.010054,0.000336,Unlabeled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339366,339367,18219.977778,7577.105556,18213,7569,18227,7587,180.0,0.866667,3.361111,...,0.155556,0.155556,0.020937,0.166667,0.372222,0.228595,1.000000,6.500000,0.217276,Epithelial
339367,339368,18219.058824,7696.029412,18215,7692,18224,7701,68.0,0.647059,0.735294,...,0.073529,0.073529,0.009897,0.000000,0.000000,0.000000,1.000000,2.500000,0.083568,Epithelial
339368,339369,18233.210300,7532.523605,18224,7525,18244,7541,233.0,0.433476,0.600858,...,0.038627,0.038627,0.005199,0.034335,0.051502,0.031629,0.828326,1.549356,0.051790,Epithelial
339369,339370,18235.310433,7561.203562,18224,7540,18247,7585,786.0,0.853690,3.506361,...,0.190840,0.202290,0.027227,0.055980,0.092875,0.057038,0.992366,7.441476,0.248747,Epithelial


In [ ]:
df, a = mip.get_spatial_features(seg_fp, ome_fp, thresholds=thresholds, gating_strategy=gating_strategy)